In [6]:
import requests
import json
import pandas as pd
from io import StringIO

from TCGA_imports.dgc_api.fields import CASES_FIELDS

In [67]:
CASES_FIELDS

# [field for field in CASES_FIELDS if field.startswith('project')]
[field for field in CASES_FIELDS if field=='primary_site']

[]

In [103]:
import requests
import json
import re

files_endpt = "https://api.gdc.cancer.gov/files"

# filters = {
#     "op": "in",
#         "content":{
#             "field": "files.data_format",
#             "value": ["MAF"]
#             }
# }

filters = {
    "op": "and",
    "content":[
        # {
        # "op": "in",
        # "content":{
        #     "field": "data_format",
        #     "value": ["MAF"]
        #     }
        # },
        # {
        # "op": "in",
        # "content":{
        #     "field": "data_type",
        #     "value": ["Masked Somatic Mutation"]
        #     }
        # },
        {
        "op": "in",
        "content":{
            "field": "data_type",
            "value": ["Clinical Supplement"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "cases.project.project_id",
            "value": ["TCGA-BRCA"]
            }
        }
    ]
}

# Here a GET is used, so the filter parameters should be passed as a JSON string.

params = {
    "filters": json.dumps(filters),
    "fields": "file_id,cases.project.project_id,data_format,data_category,data_type",
    "format": "TSV",
    "size": "10"
    }

response = requests.get(files_endpt, params = params)

out_df = pd.read_csv(StringIO(response.content.decode("utf-8")), sep='\t')

file_id = list(out_df['file_id'])

# out_df

In [104]:
out_df

,cases.0.project.project_id,data_category,data_format,data_type,file_id,id
0,TCGA-BRCA,Clinical,BCR XML,Clinical Supplement,cb48c54b-afe4-46fc-bcde-7a802c6c6157,cb48c54b-afe4-46fc-bcde-7a802c6c6157
1,TCGA-BRCA,Clinical,BCR XML,Clinical Supplement,9f870d93-c292-48ff-9b88-7d907dcbb929,9f870d93-c292-48ff-9b88-7d907dcbb929
2,TCGA-BRCA,Clinical,BCR XML,Clinical Supplement,6a09fc95-7c32-4545-8c48-a4b2027b649f,6a09fc95-7c32-4545-8c48-a4b2027b649f
3,TCGA-BRCA,Clinical,BCR XML,Clinical Supplement,745674b2-3aaa-433b-a76a-7220a8eb7e46,745674b2-3aaa-433b-a76a-7220a8eb7e46
4,TCGA-BRCA,Clinical,BCR XML,Clinical Supplement,be44103d-9aad-48bf-a727-fcdf6d625d54,be44103d-9aad-48bf-a727-fcdf6d625d54
5,TCGA-BRCA,Clinical,BCR XML,Clinical Supplement,51e0e5c5-b139-4a87-99a9-7b0058533b91,51e0e5c5-b139-4a87-99a9-7b0058533b91
6,TCGA-BRCA,Clinical,BCR XML,Clinical Supplement,93d42e70-8990-4a82-9ef0-c1775737355b,93d42e70-8990-4a82-9ef0-c1775737355b
7,TCGA-BRCA,Clinical,BCR XML,Clinical Supplement,911d0bbf-ee9a-407e-a78d-5e903d072dd6,911d0bbf-ee9a-407e-a78d-5e903d072dd6
8,TCGA-BRCA,Clinical,BCR XML,Clinical Supplement,5461b42e-2c5c-4f21-8524-4e7b5ec3a777,5461b42e-2c5c-4f21-8524-4e7b5ec3a777
9,TCGA-BRCA,Clinical,BCR XML,Clinical Supplement,5ed3dafb-8a63-4f7c-8a37-eb8700afcef8,5ed3dafb-8a63-4f7c-8a37-eb8700afcef8


In [105]:
data_endpt = "https://api.gdc.cancer.gov/data"

params = {"ids": file_id}

response = requests.post(data_endpt,
                        data = json.dumps(params),
                        headers={
                            "Content-Type": "application/json"
                            })

response_head_cd = response.headers["Content-Disposition"]

file_name = re.findall("filename=(.+)", response_head_cd)[0] + '_clinical'

with open(file_name, "wb") as output_file:
    output_file.write(response.content)

In [ ]:
import requests
import json
import re

files_endpt = "https://api.gdc.cancer.gov/cases"

# filters = {
#     "op": "and",
#     "content":[
#         {
#         "op": "in",
#         "content":{
#             "field": "cases.project.primary_site",
#             "value": ["Lung"]
#             }
#         },
#         # {
#         # "op": "in",
#         # "content":{
#         #     "field": "cases.project.primary_site",
#         #     "value": ["Kidney"]
#         #     }
#         # },
#     ]
# }

filters = {
    "op": "in",
    "content":{
        "field": "primary_site",
        "value": ["Kidney"]
        }
}

# filters = {
#     "op": "in",
#     "content":{
#         "field": "primary_site",
#         "value": ["Kidney"]
#         }
#     }

fields = [
    "submitter_id",
    "case_id",
    "primary_site",
    "disease_type",
    "vital_status"
    ]
fields = ",".join(fields)


# Here a GET is used, so the filter parameters should be passed as a JSON string.

params = {
    "filters": json.dumps(filters),
    "fields": fields,
    "format": "TSV",
    "size": "100"
    }

response = requests.get(files_endpt, params = params)

# print(response.content.decode("utf-8"))
out_df = pd.read_csv(StringIO(response.content.decode("utf-8")), sep='\t')
out_df

,case_id,disease_type,id,primary_site,submitter_id
0,63653137-c694-4ad6-83a3-eaaff1df2ca7,Complex Mixed and Stromal Neoplasms,63653137-c694-4ad6-83a3-eaaff1df2ca7,Kidney,HCM-BROD-0004-C64
1,a3b71c6f-77cc-4e80-8c1c-96cdf836b42e,Complex Mixed and Stromal Neoplasms,a3b71c6f-77cc-4e80-8c1c-96cdf836b42e,Kidney,HCM-BROD-0055-C64
2,8ed2333b-d19c-4fa1-9433-f349a91159bf,Adenomas and Adenocarcinomas,8ed2333b-d19c-4fa1-9433-f349a91159bf,Kidney,HCM-BROD-0711-C64
3,b1ed6908-e164-4e0f-accd-11605edf4c81,Ductal and Lobular Neoplasms,b1ed6908-e164-4e0f-accd-11605edf4c81,Kidney,HCM-BROD-0353-C64
4,ec082842-b5e2-4673-b1d5-68ff36d3522b,Adenomas and Adenocarcinomas,ec082842-b5e2-4673-b1d5-68ff36d3522b,Kidney,TCGA-KM-A7QB
...,...,...,...,...,...
95,4a88bc85-4c6e-4fac-8dbb-d3fc5f36b35e,Adenomas and Adenocarcinomas,4a88bc85-4c6e-4fac-8dbb-d3fc5f36b35e,Kidney,TCGA-KN-8422
96,4e4faf51-2cbf-4910-a093-186660e4a19b,Adenomas and Adenocarcinomas,4e4faf51-2cbf-4910-a093-186660e4a19b,Kidney,TCGA-KM-8476
97,6101ffe6-2ef6-4256-9d6b-a7c545836995,Adenomas and Adenocarcinomas,6101ffe6-2ef6-4256-9d6b-a7c545836995,Kidney,TCGA-KL-8325
98,648fecca-4946-4880-8359-6801cbb4fc0d,Adenomas and Adenocarcinomas,648fecca-4946-4880-8359-6801cbb4fc0d,Kidney,TCGA-KO-8403


In [36]:
import requests
import json

fields = [
    "submitter_id",
    "case_id",
    "primary_site",
    "disease_type",
    "diagnoses.vital_status"
    ]

fields = ",".join(fields)

cases_endpt = "https://api.gdc.cancer.gov/cases"

filters = {
    "op": "in",
    "content":{
        "field": "primary_site",
        "value": ["Kidney"]
        }
    }

# With a GET request, the filters parameter needs to be converted
# from a dictionary to JSON-formatted string

params = {
    "filters": json.dumps(filters),
    "fields": fields,
    "format": "TSV",
    "size": "100"
    }

response = requests.get(cases_endpt, params = params)

print(response.content)

b'case_id\tdisease_type\tid\tprimary_site\tsubmitter_id\r\n63653137-c694-4ad6-83a3-eaaff1df2ca7\tComplex Mixed and Stromal Neoplasms\t63653137-c694-4ad6-83a3-eaaff1df2ca7\tKidney\tHCM-BROD-0004-C64\r\na3b71c6f-77cc-4e80-8c1c-96cdf836b42e\tComplex Mixed and Stromal Neoplasms\ta3b71c6f-77cc-4e80-8c1c-96cdf836b42e\tKidney\tHCM-BROD-0055-C64\r\n8ed2333b-d19c-4fa1-9433-f349a91159bf\tAdenomas and Adenocarcinomas\t8ed2333b-d19c-4fa1-9433-f349a91159bf\tKidney\tHCM-BROD-0711-C64\r\nb1ed6908-e164-4e0f-accd-11605edf4c81\tDuctal and Lobular Neoplasms\tb1ed6908-e164-4e0f-accd-11605edf4c81\tKidney\tHCM-BROD-0353-C64\r\nec082842-b5e2-4673-b1d5-68ff36d3522b\tAdenomas and Adenocarcinomas\tec082842-b5e2-4673-b1d5-68ff36d3522b\tKidney\tTCGA-KM-A7QB\r\nc7679861-f8a5-4dc1-853c-943b912253f6\tAdenomas and Adenocarcinomas\tc7679861-f8a5-4dc1-853c-943b912253f6\tKidney\tTCGA-UW-A72L\r\nd0a8cfdc-c025-4de9-94ad-f5160809ff8f\tAdenomas and Adenocarcinomas\td0a8cfdc-c025-4de9-94ad-f5160809ff8f\tKidney\tTCGA-UW-A7GX\